In [3]:
html = '''
<div id="result1">
<p class=row>
<a class="red" href="/link1">링크1</a>
<a class="blue" href="/link2">링크2</a>
</p>
<p class=row>
<a class="red" href="/link1">링크1</a>
<a class="blue" href="/link2">링크2</a>
</p>
</div>
<div id="result2">
<p class=row>
<a class="red" href="/link1">링크1</a>
<a class="blue" href="/link2">링크2</a>
</p>
<p class=row>
<a class="red" href="/link1">링크1</a>
<a class="blue" href="/link2">링크2</a>
</p>
</div>
'''

In [4]:
from bs4 import BeautifulSoup

dom = BeautifulSoup(html, 'html5lib')

In [5]:
div = dom.find(attrs={'id':'result1'})
p = div.find(recursive=False)
p.find_all(recursive=False)

[<a class="red" href="/link1">링크1</a>, <a class="blue" href="/link2">링크2</a>]

In [6]:
div = dom.select_one('#result1')
p = div.select_one('p')
p.select('a')

[<a class="red" href="/link1">링크1</a>, <a class="blue" href="/link2">링크2</a>]

In [7]:
dom.select('#result1 > p > a')
# id=result1
# <p> 자식
#  <a></a>
#  <a></a>
# </p>
# <p> 자식
#  <a></a>
#  <a></a>
# </p>

[<a class="red" href="/link1">링크1</a>,
 <a class="blue" href="/link2">링크2</a>,
 <a class="red" href="/link1">링크1</a>,
 <a class="blue" href="/link2">링크2</a>]

In [8]:
dom.select('#result1 > p:first-child > a')
dom.select('#result1 > p:nth-child(1) > a')

[<a class="red" href="/link1">링크1</a>, <a class="blue" href="/link2">링크2</a>]

In [9]:
# +; next_sibling
# p + p(*)
# (*)p:has(+ p)
dom.select('#result1 > p:has(+ p) > a')
dom.select('#result1 > p + p > a')

[<a class="red" href="/link1">링크1</a>, <a class="blue" href="/link2">링크2</a>]

In [10]:
dom.select('#result1 > p:has(+ p) > a')[0]\
 is dom.select('#result1 > p + p > a')[0]

False

In [13]:
dom.select('article a')

[]

In [14]:
from requests import request
from requests.exceptions import HTTPError
from time import sleep

def download(url, params={}, method='GET', retries=3):
    resp = None
    
    try:
        resp = request(method, url,
                       params=params if method=='GET' else {},
                       data=params if method=='POST' else {},
                       headers={'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36'})
        resp.raise_for_status()
        # 아니면,
        # if resp.status_code != 200:
    except HTTPError as e:
        if 500 <= e.response.status_code:
            if retries > 0:
                sleep(3)
                resp = download(url, params=params,
                                method=method,
                                retries=retries-1)
            else:
                print('재방문 횟수 초과')
        else:
            print('Request', resp.request.headers)
            print('Response', e.response.headers)
        
    return resp